In [1]:
import sys
sys.path.insert(0, "/export/servers/wenhao/database_reader/")

%load_ext autoreload
%autoreload 2

In [2]:
import database_reader as dr

/usr/local/python-3.6.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/python-3.6.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/python-3.6.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/python-3.6.8/lib/python3.6/site-packages/tensorflow/python/framework

In [3]:
import database_reader.utils as du

In [4]:
from database_reader import CPSC2018

In [5]:
hehe_cpsc = CPSC2018(db_path="/export/servers/data/CPSC2018/Training_WFDB/", verbose=5)

In [ ]:
ro = 608

In [ ]:
hehe_cpsc.load_ann(ro)

In [ ]:
hehe_cpsc.load_data(ro)[:,0]

In [ ]:
hehe_cpsc.load_ann(1)['df_leads']['lead_name'].tolist()

In [ ]:
for ro in range(1, hehe_cpsc.nb_records+1):
    try:
        ll = hehe_cpsc.load_ann(ro)['df_leads']['lead_name'].tolist()
        if ll != ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']:
            print(f"第{ro}个record的导联顺序不一样")
    except Exception as e:
        print(f"第{ro}个record发生了如下错误")
        print(e)

In [ ]:
for ro in range(1, hehe_cpsc.nb_records+1):
    try:
        ll = hehe_cpsc.load_data(ro)[:,0]
        fv = hehe_cpsc.load_ann(ro)['df_leads']['first_value'].values
        if (ll!=fv).any():
            print(f"第{ro}个record的导联顺序不一样")
    except Exception as e:
        print(f"第{ro}个record发生了如下错误")
        print(e)

In [ ]:
hehe_cpsc.get_labels(ro)

In [ ]:
hehe_cpsc.get_diagnosis(ro)

In [ ]:
hehe_cpsc.load_data(ro).shape

In [ ]:
hehe_cpsc.plot(ro)

In [15]:
hehe_cpsc.load_data(1).shape[0]//3000

2

In [10]:
input_len = 6*hehe_cpsc.freq

In [12]:
x,y = [], []

In [16]:
for ro in range(1, hehe_cpsc.nb_records+1):
    ecg = hehe_cpsc.load_data(ro)
    ann = hehe_cpsc.get_labels(ro)
    for i in range(ecg.shape[0]//input_len):
        x.append(ecg[input_len*i:input_len*(i+1)])
        y.append(ann)

In [25]:
((x[0]-np.mean(x[0]))/np.std(x[0])).shape

(3000, 12)

In [26]:
new_x = [(item-np.mean(item))/np.std(item) for item in x]

In [27]:
new_x = np.array(new_x)

In [28]:
new_x.shape

(14579, 3000, 12)

In [29]:
y = np.array(y)

In [32]:
np.save('./cpsc_y.npy', y)

In [33]:
np.save('./cpsc_x.npy', new_x)

In [ ]:
np.array([[1,1,10],[2,3,50]]) - np.array([])

In [45]:
hehe_cpsc.all_diagnosis

['N', 'AF', 'I-AVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD', 'STE']

In [47]:
new_y = np.zeros((new_x.shape[0],9))
new_y

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [48]:
for idx, item in enumerate(y):
    indices = [hehe_cpsc.all_diagnosis.index(l) for l in item]
    new_y[idx][indices] = 1

In [51]:
new_y[0],y[0], hehe_cpsc.all_diagnosis

(array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]),
 ['RBBB'],
 ['N', 'AF', 'I-AVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD', 'STE'])

In [34]:
train_x = new_x[:12000]
test_x = new_x[12000:]

In [35]:
train_x.shape

(12000, 3000, 12)

In [58]:
train_y.shape

(12000, 9)

In [52]:
train_y = new_y[:12000]
test_y = new_y[12000:]

In [89]:
?BatchNormalization

In [53]:
from keras import layers
from keras import Input
from keras.models import Sequential, Model, load_model
from keras.layers import (
    LSTM, GRU,
    TimeDistributed, Bidirectional,
    ReLU, LeakyReLU,
    BatchNormalization,
    Dense, Dropout, Activation, Flatten, 
    Input, Reshape, GRU, CuDNNGRU,
    Conv1D,
    MaxPooling1D,
    GlobalAveragePooling1D, AveragePooling1D,
    concatenate,
)
from keras.initializers import he_normal, he_uniform, Orthogonal
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint


SEED = 42
freq = 500
cell_len_t = 6
nb_classes = 9
batch_size=128


TI_CNN = Sequential(name='TI_CNN')

TI_CNN.add(
    Conv1D(
        input_shape = (freq*cell_len_t, 12),
        filters=64, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=64, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)
TI_CNN.add(
    Conv1D(
        filters=128, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=128, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)
TI_CNN.add(
    Conv1D(
        filters=256, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=256, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)

TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=256, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)

TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)

TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)

TI_CNN.add(
    LSTM(
        128, kernel_initializer=Orthogonal(seed=SEED),
        return_sequences=True,
    )
)

TI_CNN.add(
    LSTM(
        32, kernel_initializer=Orthogonal(seed=SEED),
        return_sequences=True,
    )
)

TI_CNN.add(
    LSTM(
        9, kernel_initializer=Orthogonal(seed=SEED),
        return_sequences=False,
    )
)

TI_CNN.add(
    Dense(nb_classes,activation='sigmoid')
)


In [ ]:
TI_CNN.summary()

Model: "TI_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_27 (Conv1D)           (None, 3000, 64)          2368      
_________________________________________________________________
batch_normalization_27 (Batc (None, 3000, 64)          256       
_________________________________________________________________
re_lu_27 (ReLU)              (None, 3000, 64)          0         
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 3000, 64)          12352     
_________________________________________________________________
batch_normalization_28 (Batc (None, 3000, 64)          256       
_________________________________________________________________
re_lu_28 (ReLU)              (None, 3000, 64)          0         
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 1000, 64)          0    

In [ ]:

TI_CNN.compile(loss='binary_crossentropy', optimizer=Adam(0.0001))

checkpointer = ModelCheckpoint(filepath='/tmp/weights.hdf5', verbose=1, monitor='val_acc', save_best_only=True)


In [ ]:


TI_CNN.fit(train_x, train_y, batch_size=128, epochs=200, verbose=1, validation_data=(test_x, test_y), callbacks=[checkpointer])

Instructions for updating:
Use tf.cast instead.
Train on 12000 samples, validate on 2579 samples
Epoch 1/200
12000/12000 [==============================] - 310s 26ms/step - loss: 0.5346 - val_loss: 0.4976

Epoch 00001: saving model to /tmp/weights.hdf5
Epoch 2/200
12000/12000 [==============================] - 205s 17ms/step - loss: 0.4627 - val_loss: 0.4511

Epoch 00002: saving model to /tmp/weights.hdf5
Epoch 3/200
12000/12000 [==============================] - 203s 17ms/step - loss: 0.4287 - val_loss: 0.4281

Epoch 00003: saving model to /tmp/weights.hdf5
Epoch 4/200
12000/12000 [==============================] - 206s 17ms/step - loss: 0.4039 - val_loss: 0.4149

Epoch 00004: saving model to /tmp/weights.hdf5
Epoch 5/200
12000/12000 [==============================] - 208s 17ms/step - loss: 0.3851 - val_loss: 0.3939

Epoch 00005: saving model to /tmp/weights.hdf5
Epoch 6/200
12000/12000 [==============================] - 205s 17ms/step - loss: 0.3688 - val_loss: 0.3813

Epoch 00006: s

In [59]:
submitted_model = load_model("./tmp/weights-0.22loss.hdf5")

In [61]:
latest_model = load_model("./tmp/weights.hdf5")

In [67]:
def compute_beta_score(labels, output, beta, num_classes, check_errors=True):

    # Check inputs for errors.
    if check_errors:
        if len(output) != len(labels):
            raise Exception('Numbers of outputs and labels must be the same.')

    # Populate contingency table.
    num_recordings = len(labels)

    fbeta_l = np.zeros(num_classes)
    gbeta_l = np.zeros(num_classes)
    fmeasure_l = np.zeros(num_classes)
    accuracy_l = np.zeros(num_classes)

    f_beta = 0
    g_beta = 0
    f_measure = 0
    accuracy = 0

    # Weight function
    C_l=np.ones(num_classes);

    for j in range(num_classes):
        tp = 0
        fp = 0
        fn = 0
        tn = 0

        for i in range(num_recordings):
            
            num_labels = np.sum(labels[i])
        
            if labels[i][j] and output[i][j]:
                tp += 1/num_labels
            elif not labels[i][j] and output[i][j]:
                fp += 1/num_labels
            elif labels[i][j] and not output[i][j]:
                fn += 1/num_labels
            elif not labels[i][j] and not output[i][j]:
                tn += 1/num_labels

        # Summarize contingency table.
        if ((1+beta**2)*tp + (fn*beta**2) + fp):
            fbeta_l[j] = float((1+beta**2)* tp) / float(((1+beta**2)*tp) + (fn*beta**2) + fp)
        else:
            fbeta_l[j] = 1.0

        if (tp + fp + beta * fn):
            gbeta_l[j] = float(tp) / float(tp + fp + beta*fn)
        else:
            gbeta_l[j] = 1.0

        if tp + fp + fn + tn:
            accuracy_l[j] = float(tp + tn) / float(tp + fp + fn + tn)
        else:
            accuracy_l[j] = 1.0

        if 2 * tp + fp + fn:
            fmeasure_l[j] = float(2 * tp) / float(2 * tp + fp + fn)
        else:
            fmeasure_l[j] = 1.0


    for i in range(num_classes):
        f_beta += fbeta_l[i]*C_l[i]
        g_beta += gbeta_l[i]*C_l[i]
        f_measure += fmeasure_l[i]*C_l[i]
        accuracy += accuracy_l[i]*C_l[i]


    f_beta = float(f_beta)/float(num_classes)
    g_beta = float(g_beta)/float(num_classes)
    f_measure = float(f_measure)/float(num_classes)
    accuracy = float(accuracy)/float(num_classes)


    return accuracy,f_measure,f_beta,g_beta

In [82]:
y_pred = submitted_model.predict(train_x)

In [63]:
test_y.shape

(2579, 9)

In [64]:
test_x.shape

(2579, 3000, 12)

In [66]:
hehe_y = test_y[:1300]
hehe_x = test_x[:1300]

hehe_y.shape, hehe_x.shape

((1300, 9), (1300, 3000, 12))

In [75]:
y_pred_s = submitted_model.predict(test_x)

In [76]:
y_pred_l = latest_model.predict(test_x)

In [77]:
y_pred_s.shape, y_pred_l.shape

((2579, 9), (2579, 9))

In [87]:
import numpy as np, os, os.path, sys


def evaluate_12ECG_score(label_directory, output_directory):

    # Set parameters.
    label_header       = '12ECGLabel'
    output_label_header  = 'OutputLabel'
    output_probability_header = 'OutputProbability'

    beta = 2
    labels=[]
    output=[]
    output_probabilities=[]

    # Find label and output files.
    label_files = []
    for f in os.listdir(label_directory):
        g = os.path.join(label_directory, f)
        if os.path.isfile(g) and not f.lower().startswith('.') and f.lower().endswith('hea'):
            label_files.append(g)
    label_files = sorted(label_files)

    output_files = []
    for f in os.listdir(output_directory):
        g = os.path.join(output_directory, f)
        if os.path.isfile(g) and not f.lower().startswith('.') and f.lower().endswith('csv'):
            output_files.append(g)
    output_files = sorted(output_files)

    if len(label_files) != len(output_files):
        raise Exception('Numbers of label and output files must be the same.')

    classes = get_classes(label_files)


    # Load labels and outputs.
    num_files = len(label_files)

    for k in range(num_files):

        recording_label,classes_label,single_recording_labels=get_true_labels(label_files[k],classes)
        
        with open(output_files[k],'r') as f:
            tmp_data = f.readlines()
        recording_output = tmp_data[0]
        classes_output = tmp_data[1].split(',')
        single_recording_output = np.array(tmp_data[2].split(','),np.int)
        single_probabilities_output = np.array(tmp_data[3].split(','),np.float64)

       # Check labels and output for errors.

        if not (len(classes_label) == len(classes_output)):
            raise Exception('Numbers of classes for a file must be the same.')
        
        if not (len(single_recording_labels) == len(single_recording_output) == len(single_probabilities_output)):
            raise Exception('Numbers of labels and output for a file must be the same.')

        labels.append(single_recording_labels)
        output.append(single_recording_output)
        output_probabilities.append(single_probabilities_output)

    labels=np.array(labels)
    output=np.array(output)
    output_probabilities=np.array(output_probabilities)

    num_classes = len(classes_label)

    # Compute F_beta measure and the generalization of the Jaccard index
    accuracy,f_measure,Fbeta_measure,Gbeta_measure = compute_beta_score(labels, output, beta, num_classes)

    # compute AUROC and AUPRC
    auroc,auprc = compute_auc(labels, output_probabilities,num_classes)

    return auroc,auprc,accuracy,f_measure,Fbeta_measure,Gbeta_measure

# Find unique number of classes
def get_classes(files):

    classes=set()
    for input_file in files:
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
                        classes.add(c.strip())

    return sorted(classes)



# Find unique true labels
def get_true_labels(input_file,classes):

    classes_label = classes
    single_recording_labels=np.zeros(len(classes),dtype=int)


    with open(input_file,'r') as f:
        first_line = f.readline()
        recording_label=first_line.split(' ')[0]
        print(recording_label)
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                for c in tmp:
                    idx = classes.index(c.strip())
                    single_recording_labels[idx]=1

    return recording_label,classes_label,single_recording_labels

def compute_auc(labels, probabilities, num_classes, check_errors=True):


    # Check inputs for errors.
    if check_errors:
        if len(labels) != len(probabilities):
            raise Exception('Numbers of outputs and labels must be the same.')

    find_NaNs = np.isnan(probabilities);
    probabilities[find_NaNs] = 0;

    auroc_l = np.zeros(num_classes)
    auprc_l = np.zeros(num_classes)

    auroc = 0
    auprc = 0

    # Weight function - this will change
    C_l=np.ones(num_classes);

    # Populate contingency table.
    num_recordings = len(labels)

    for k in range(num_classes):
    

            # Find probabilities thresholds.
        thresholds = np.unique(probabilities[:,k])[::-1]
        if thresholds[0] != 1:
            thresholds = np.insert(thresholds, 0, 1)
        if thresholds[-1] == 0:
            thresholds = thresholds[:-1]

        m = len(thresholds)
    

        # Populate contingency table across probabilities thresholds.
        tp = np.zeros(m)
        fp = np.zeros(m)
        fn = np.zeros(m)
        tn = np.zeros(m)

        # Find indices that sort the predicted probabilities from largest to
        # smallest.
        idx = np.argsort(probabilities[:,k])[::-1]

        i = 0
        for j in range(m):
            # Initialize contingency table for j-th probabilities threshold.
            if j == 0:
                tp[j] = 0
                fp[j] = 0
                fn[j] = np.sum(labels[:,k])
                tn[j] = num_recordings - fn[j]
            else:
                tp[j] = tp[j - 1]
                fp[j] = fp[j - 1]
                fn[j] = fn[j - 1]
                tn[j] = tn[j - 1]
            # Update contingency table for i-th largest predicted probability.
            while i < num_recordings and probabilities[idx[i],k] >= thresholds[j]:
                if labels[idx[i],k]:
                    tp[j] += 1
                    fn[j] -= 1
                else:
                    fp[j] += 1
                    tn[j] -= 1
                i += 1

        # Summarize contingency table.
        tpr = np.zeros(m)
        tnr = np.zeros(m)
        ppv = np.zeros(m)
        npv = np.zeros(m)


        for j in range(m):
            if tp[j] + fn[j]:
                tpr[j] = float(tp[j]) / float(tp[j] + fn[j])
            else:
                tpr[j] = 1
            if fp[j] + tn[j]:
                tnr[j] = float(tn[j]) / float(fp[j] + tn[j])
            else:
                tnr[j] = 1
            if tp[j] + fp[j]:
                ppv[j] = float(tp[j]) / float(tp[j] + fp[j])
            else:
                ppv[j] = 1
            if fn[j] + tn[j]:
                npv[j] = float(tn[j]) / float(fn[j] + tn[j])
            else:
                npv[j] = 1

        # Compute AUROC as the area under a piecewise linear function with TPR /
        # sensitivity (x-axis) and TNR / specificity (y-axis) and AUPRC as the area
        # under a piecewise constant with TPR / recall (x-axis) and PPV / precision
        # (y-axis).

        for j in range(m-1):
            auroc_l[k] += 0.5 * (tpr[j + 1] - tpr[j]) * (tnr[j + 1] + tnr[j])
            auprc_l[k] += (tpr[j + 1] - tpr[j]) * ppv[j + 1]


    for i in range(num_classes):
        auroc += auroc_l[i]*C_l[i]
        auprc += auprc_l[i]*C_l[i]

    auroc = float(auroc)/float(num_classes)
    auprc = float(auprc)/float(num_classes)

    
    return auroc, auprc

In [88]:
evaluate_12ECG_score("./tmp_data/input_labels/", "./tmp_data/output_labels/")

A6000
A6001
A6002
A6003
A6004
A6005
A6006
A6007
A6008
A6009
A6010
A6011
A6012
A6013
A6014
A6015
A6016
A6017
A6018
A6019
A6020
A6021
A6022
A6023
A6024
A6025
A6026
A6027
A6028
A6029
A6030
A6031
A6032
A6033
A6034
A6035
A6036
A6037
A6038
A6039
A6040
A6041
A6042
A6043
A6044
A6045
A6046
A6047
A6048
A6049
A6050
A6051
A6052
A6053
A6054
A6055
A6056
A6057
A6058
A6059
A6060
A6061
A6062
A6063
A6064
A6065
A6066
A6067
A6068
A6069
A6070
A6071
A6072
A6073
A6074
A6075
A6076
A6077
A6078
A6079
A6080
A6081
A6082
A6083
A6084
A6085
A6086
A6087
A6088
A6089
A6090
A6091
A6092
A6093
A6094
A6095
A6096
A6097
A6098
A6099
A6100
A6101
A6102
A6103
A6104
A6105
A6106
A6107
A6108
A6109
A6110
A6111
A6112
A6113
A6114
A6115
A6116
A6117
A6118
A6119
A6120
A6121
A6122
A6123
A6124
A6125
A6126
A6127
A6128
A6129
A6130
A6131
A6132
A6133
A6134
A6135
A6136
A6137
A6138
A6139
A6140
A6141
A6142
A6143
A6144
A6145
A6146
A6147
A6148
A6149
A6150
A6151
A6152
A6153
A6154
A6155
A6156
A6157
A6158
A6159
A6160
A6161
A6162
A6163
A6164
A6165
A616

(0.8692422110291568,
 0.6546668843423026,
 0.9297208538587848,
 0.5935410020499682,
 0.5764341435248892,
 0.37287823676306764)